In [1]:
%cd ..

/home/dawood/Workspace/compoundGAN


In [2]:
import torch
import networkx as nx
import numpy as np
from tqdm import tqdm

from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Fingerprints import FingerprintMols
from PIL import Image

from data.zinc import mol2graph, Library, ZINC250K

from models.encoder import Encoder
from models.generator import Generator

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

%load_ext line_profiler


## Generator test

In [3]:
gen = Generator(64, [44, 7, 3, 3, 2], [5, 2, 2, 4], 4)
gen = gen.cuda()
gen = gen.eval()

In [9]:
x = torch.randn((32, 64)).cuda()
%lprun -f gen.forward G = gen(x)

Timer unit: 1e-06 s

Total time: 0.269614 s
File: /home/dawood/Workspace/compoundGAN/models/generator.py
Function: forward at line 41

Line #      Hits         Time  Per Hit   % Time  Line Contents
    41                                               def forward(self, z, max_nodes=50):
    42                                           
    43         1          8.0      8.0      0.0          batch_size = z.shape[0]
    44                                           
    45         1       2480.0   2480.0      0.9          output_graphs = [dgl.DGLGraph() for _ in range(batch_size)]
    46         1        317.0    317.0      0.1          num_nodes     = -1*torch.ones(batch_size)
    47         1        119.0    119.0      0.0          num_nodes     = num_nodes.cuda() if z.is_cuda else num_nodes
    48                                           
    49         1          4.0      4.0      0.0          ctx = []
    50         5         42.0      8.4      0.0          for l in self.latent_proj